In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [2]:
veri_dizini = "Data"

# Sınıfları sırala
siniflar = sorted([
    d for d in os.listdir(veri_dizini)
    if os.path.isdir(os.path.join(veri_dizini, d))
])


In [3]:
print("Sınıflar:", siniflar)

Sınıflar: ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']


In [4]:
goruntuler = []
etiketler = []

# Görüntü alımı diğer kodlarla tamamen aynı
for etiket, sinif_adi in enumerate(siniflar):
    klasor_yolu = os.path.join(veri_dizini, sinif_adi)
    dosyalar = os.listdir(klasor_yolu)
    
    for dosya in dosyalar:
        dosya_yolu = os.path.join(klasor_yolu, dosya)
        img = cv2.imread(dosya_yolu)
        if img is None:
            continue
            #Grayscale
        gri = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        yeniden_boyutlu = cv2.resize(gri, (64, 64))
        normalize_edilmis = yeniden_boyutlu / 255.0
        goruntuler.append(normalize_edilmis)
        etiketler.append(etiket)

goruntuler = np.array(goruntuler)
etiketler = np.array(etiketler)

In [5]:
print("Toplam görüntü sayısı:", len(goruntuler))

Toplam görüntü sayısı: 14248


In [6]:
import numpy as np
import cv2

def gabor_filtreleri_olustur():
    filtreler = []
    filtre_boyutu = 31  # Gabor çekirdeği boyutu

    for aci in np.arange(0, np.pi, np.pi / 4):  # 0°, 45°, 90°, 135°
        for sigma in (1, 3):  # düşük ve yüksek sigma değerleri
            for lamda in np.arange(np.pi / 4, np.pi, np.pi / 4):  # dalga boyları
                cekirdek = cv2.getGaborKernel(
                    (filtre_boyutu, filtre_boyutu),
                    sigma,
                    aci,
                    lamda,
                    gamma=0.5,
                    psi=0
                )
                filtreler.append(cekirdek)
    return filtreler

def gabor_ozellik_cikar(goruntuler, filtreler):
    ozellikler = []

    for img in goruntuler:
        img_ozellik = []
        for filtre in filtreler:
            uygulanmis = cv2.filter2D(img, cv2.CV_8UC3, filtre)
            ortalama = uygulanmis.mean()
            std = uygulanmis.std()
            img_ozellik.extend([ortalama, std])
        ozellikler.append(img_ozellik)

    return np.array(ozellikler)


In [7]:
gabor_filtreler = gabor_filtreleri_olustur()
gabor_ozellikleri = gabor_ozellik_cikar(goruntuler, gabor_filtreler)


In [8]:
# Gabor öznitelikleri ile eğitim ve test ayrımı
X_train, X_test, y_train, y_test = train_test_split(
    gabor_ozellikleri, etiketler,
    test_size=0.2,
    random_state=42,
    stratify=etiketler
)

print("Eğitim seti boyutu:", X_train.shape)
print("Test seti boyutu:", X_test.shape)


Eğitim seti boyutu: (11398, 48)
Test seti boyutu: (2850, 48)


In [9]:
#Diğer train dosylarındakiyle aynı sırada ilerleyeceğim SVM ,MLP,Random Forest ardından da XGBoost

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import time

# Denenecek SVM parametre kombinasyonları
svm_modeller = [
    {"C": 0.1, "gamma": "scale"},
    {"C": 1, "gamma": "scale"},
    {"C": 10, "gamma": "scale"},
    {"C": 1, "gamma": 0.1},
    {"C": 1, "gamma": 0.01},
    {"C": 1, "gamma": 0.001},
    {"C": 10, "gamma": 0.1},
    {"C": 10, "gamma": 0.01},
    {"C": 100, "gamma": 0.01},
    {"C": 100, "gamma": 0.001},
]

# Her model için eğitim ve test
for idx, parametreler in enumerate(svm_modeller, 1):
    print(f"\n--- Gabor + SVM Model {idx}: C={parametreler['C']}, gamma={parametreler['gamma']} ---")

    # Eğitim süresi başlangıcı
    egitim_basla = time.time()

    svm_model = SVC(
        C=parametreler["C"],
        gamma=parametreler["gamma"],
        kernel="rbf"
    )
    svm_model.fit(X_train, y_train)

    egitim_bitis = time.time()

    # Test süresi başlangıcı
    test_basla = time.time()
    y_tahmin = svm_model.predict(X_test)
    test_bitis = time.time()

    # Doğruluk ve rapor
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin, zero_division=0))

    # Süre bilgisi
    print(f"Eğitim süresi: {(egitim_bitis - egitim_basla):.2f} saniye")
    print(f"Test süresi: {(test_bitis - test_basla):.2f} saniye")



--- Gabor + SVM Model 1: C=0.1, gamma=scale ---
Doğruluk: 0.3452631578947368
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       263
           1       0.34      0.51      0.41       748
           2       0.33      0.33      0.33       805
           3       0.36      0.42      0.39       787
           4       0.00      0.00      0.00       247

    accuracy                           0.35      2850
   macro avg       0.21      0.25      0.23      2850
weighted avg       0.28      0.35      0.31      2850

Eğitim süresi: 3.56 saniye
Test süresi: 2.15 saniye

--- Gabor + SVM Model 2: C=1, gamma=scale ---
Doğruluk: 0.3607017543859649
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       263
           1       0.35      0.59      0.44       748
           2       0.36      0.28      0.32       805
           3       0.37      0.46      0.41       787
           4       0.00      0.00 

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

# Denenecek MLP parametreleri
mlp_modeller = [
    {"hidden_layer_sizes": (100,), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (100, 50), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (200,), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (100,), "learning_rate_init": 0.01},
    {"hidden_layer_sizes": (100, 50, 25), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (150,), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (256, 128), "learning_rate_init": 0.0005},
    {"hidden_layer_sizes": (128, 64, 32), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (64, 64, 64), "learning_rate_init": 0.005},
    {"hidden_layer_sizes": (512, 256, 128, 64), "learning_rate_init": 0.001},
]

# Her model için döngü
for idx, parametreler in enumerate(mlp_modeller, 1):
    print(f"\n--- Gabor + MLP Model {idx}: Katmanlar={parametreler['hidden_layer_sizes']}, Öğrenme oranı={parametreler['learning_rate_init']} ---")

    # Eğitim süresi başlangıcı
    egitim_basla = time.time()
    
    mlp_model = MLPClassifier(
        hidden_layer_sizes=parametreler["hidden_layer_sizes"],
        learning_rate_init=parametreler["learning_rate_init"],
        max_iter=500,
        random_state=42
    )
    mlp_model.fit(X_train, y_train)
    
    egitim_bitis = time.time()
    
    # Test süresi başlangıcı
    test_basla = time.time()
    y_tahmin = mlp_model.predict(X_test)
    test_bitis = time.time()
    
    # Doğruluk ve sınıflandırma raporu
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin, zero_division=0))
    
    # Süre bilgileri
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")



--- Gabor + MLP Model 1: Katmanlar=(100,), Öğrenme oranı=0.001 ---
Doğruluk: 0.4017543859649123
              precision    recall  f1-score   support

           0       0.29      0.11      0.16       263
           1       0.43      0.55      0.48       748
           2       0.39      0.31      0.34       805
           3       0.39      0.45      0.42       787
           4       0.43      0.41      0.42       247

    accuracy                           0.40      2850
   macro avg       0.38      0.37      0.37      2850
weighted avg       0.39      0.40      0.39      2850

Eğitim süresi: 3.00 saniye
Test süresi: 0.00 saniye

--- Gabor + MLP Model 2: Katmanlar=(100, 50), Öğrenme oranı=0.001 ---
Doğruluk: 0.387719298245614
              precision    recall  f1-score   support

           0       0.31      0.14      0.19       263
           1       0.38      0.70      0.50       748
           2       0.38      0.29      0.33       805
           3       0.42      0.27      0.33   

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

# Denenecek Random Forest parametreleri
rf_modeller = [
    {"n_estimators": 100, "max_depth": None},
    {"n_estimators": 100, "max_depth": 10},
    {"n_estimators": 200, "max_depth": None},
    {"n_estimators": 200, "max_depth": 20},
    {"n_estimators": 300, "max_depth": 15},
    {"n_estimators": 500, "max_depth": None},
]

# Her model için döngü
for idx, parametreler in enumerate(rf_modeller, 1):
    print(f"\n--- Gabor + Random Forest Model {idx}: n_estimators={parametreler['n_estimators']}, max_depth={parametreler['max_depth']} ---")
    
    # Eğitim süresi başlangıcı
    egitim_basla = time.time()
    
    rf_model = RandomForestClassifier(
        n_estimators=parametreler["n_estimators"],
        max_depth=parametreler["max_depth"],
        random_state=42
    )
    rf_model.fit(X_train, y_train)
    
    egitim_bitis = time.time()
    
    # Test süresi başlangıcı
    test_basla = time.time()
    y_tahmin = rf_model.predict(X_test)
    test_bitis = time.time()
    
    # Doğruluk ve sınıflandırma raporu
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin, zero_division=0))
    
    # Süre bilgileri
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")



--- Gabor + Random Forest Model 1: n_estimators=100, max_depth=None ---
Doğruluk: 0.3849122807017544
              precision    recall  f1-score   support

           0       0.28      0.13      0.17       263
           1       0.39      0.48      0.43       748
           2       0.37      0.40      0.38       805
           3       0.40      0.42      0.41       787
           4       0.40      0.21      0.28       247

    accuracy                           0.38      2850
   macro avg       0.37      0.33      0.34      2850
weighted avg       0.38      0.38      0.38      2850

Eğitim süresi: 4.31 saniye
Test süresi: 0.04 saniye

--- Gabor + Random Forest Model 2: n_estimators=100, max_depth=10 ---
Doğruluk: 0.3792982456140351
              precision    recall  f1-score   support

           0       0.32      0.11      0.16       263
           1       0.37      0.50      0.43       748
           2       0.37      0.36      0.36       805
           3       0.40      0.45      0

In [13]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

# Denenecek XGBoost parametreleri
xgb_modeller = [
    {"max_depth": 3, "learning_rate": 0.1, "n_estimators": 100},
    {"max_depth": 5, "learning_rate": 0.1, "n_estimators": 200},
    {"max_depth": 7, "learning_rate": 0.05, "n_estimators": 200},
    {"max_depth": 5, "learning_rate": 0.2, "n_estimators": 300},
    {"max_depth": 4, "learning_rate": 0.01, "n_estimators": 500},
    {"max_depth": 6, "learning_rate": 0.1, "n_estimators": 100},
]

# Her model için eğitim ve test süreci
for idx, parametreler in enumerate(xgb_modeller, 1):
    print(f"\n--- Gabor + XGBoost Model {idx}: max_depth={parametreler['max_depth']}, learning_rate={parametreler['learning_rate']}, n_estimators={parametreler['n_estimators']} ---")
    
    # Eğitim süresi başlangıcı
    egitim_basla = time.time()
    
    xgb_model = XGBClassifier(
        max_depth=parametreler["max_depth"],
        learning_rate=parametreler["learning_rate"],
        n_estimators=parametreler["n_estimators"],
        eval_metric='mlogloss',
        random_state=42
    )
    xgb_model.fit(X_train, y_train)
    
    # Eğitim süresi bitişi
    egitim_bitis = time.time()
    
    # Test süresi başlangıcı
    test_basla = time.time()
    y_tahmin = xgb_model.predict(X_test)
    test_bitis = time.time()
    
    # Doğruluk ve raporlama
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin, zero_division=0))
    
    # Süre bilgileri
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")



--- Gabor + XGBoost Model 1: max_depth=3, learning_rate=0.1, n_estimators=100 ---
Doğruluk: 0.3778947368421053
              precision    recall  f1-score   support

           0       0.29      0.13      0.18       263
           1       0.39      0.52      0.45       748
           2       0.36      0.33      0.34       805
           3       0.38      0.42      0.40       787
           4       0.38      0.23      0.29       247

    accuracy                           0.38      2850
   macro avg       0.36      0.33      0.33      2850
weighted avg       0.37      0.38      0.37      2850

Eğitim süresi: 0.62 saniye
Test süresi: 0.00 saniye

--- Gabor + XGBoost Model 2: max_depth=5, learning_rate=0.1, n_estimators=200 ---
Doğruluk: 0.3894736842105263
              precision    recall  f1-score   support

           0       0.27      0.14      0.19       263
           1       0.42      0.49      0.45       748
           2       0.37      0.38      0.38       805
           3      

In [15]:
#from sklearn.neural_network import MLPClassifier
#import joblib

# Modeli tanımla aşaması yukardaki en yüksek başarıyı veren model bu
#gabor_mlp_model = MLPClassifier(
 #   hidden_layer_sizes=(150,),
  #  activation='relu',
   # solver='adam',
   # learning_rate_init=0.001,
   # max_iter=300,
    #random_state=42
#)

# Modeli eğitim
#gabor_mlp_model.fit(X_train, y_train)

# Kaydetme kısmı
#joblib.dump(gabor_mlp_model, "model_gabor_mlp.pkl")

#print("Gabor + MLP (Model 6) başarıyla kaydedildi.")
